In [1]:
import tensorflow as tf
from datetime import datetime

In [2]:
IMAGE_SIZE = 112
INPUT_SIZE = 96
DST_INPUT_SIZE = 56
NUM_CLASS = 5
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 5000
BATCH_SIZE = 100
LOGDIR='/tmp/data.%s' % datetime.now().isoformat()

KINDS = {
    0: 'バッグ',
    1: 'パンツ',
    2: 'シューズ',
    3: 'スカート',
    4: 'トップス'
}

print(LOGDIR)

def inference(images, keep_prob):
    
    x_image = tf.reshape(images, [-1, DST_INPUT_SIZE, DST_INPUT_SIZE, 3])

    with tf.name_scope('filter1'):
        
        num_filters1 = 32

        W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 3, num_filters1], stddev = 0.1))
        b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))
        h_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME')
    
    with tf.name_scope('pool1'):
        h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)
        h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    with tf.name_scope('filter2'):
        num_filters2 = 64

        W_conv2 = tf.Variable( tf.truncated_normal([5,5, num_filters1, num_filters2],stddev=0.1)) 
        b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
        h_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1,1,1,1], padding='SAME')
    
    with tf.name_scope('pool2'):
        h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)
        h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
    with tf.name_scope('filter3'):
        num_filters3 = 128
        
        W_conv3 = tf.Variable(tf.truncated_normal([5, 5, num_filters2, num_filters3], stddev=0.1))
        b_conv3 = tf.Variable(tf.constant(0.1, shape=[num_filters3]))
        h_conv3 = tf.nn.conv2d(h_pool2, W_conv3, strides=[1, 1, 1, 1], padding='SAME')
        
    with tf.name_scope('pool3'):
        h_conv3_cutoff = tf.nn.relu(h_conv3 + b_conv3)
        h_pool3 = tf.nn.max_pool(h_conv3_cutoff, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    with tf.name_scope('fc1'):
        w = int(DST_INPUT_SIZE / 8)
        h_pool3_flat = tf.reshape(h_pool3, [-1, w*w*num_filters3])

        num_units1 = w*w*num_filters3
        num_units2 = 1024

        w3 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
        b3 = tf.Variable(tf.constant(0.1, shape=[num_units2])) 
        hidden3 = tf.nn.relu(tf.matmul(h_pool3_flat, w3) + b3)
        
        hidden3_drop = tf.nn.dropout(hidden3, keep_prob)
        
        w0 = tf.Variable(tf.zeros([num_units2, 5]))
        b0 = tf.Variable(tf.zeros([5]))
        
        tf.summary.histogram('weights_output', w0)
        tf.summary.histogram('biases_output', b0)
        
    with tf.name_scope('sotmax'):
        p = tf.nn.softmax(tf.matmul(hidden3_drop, w0) + b0)
        
    return p

/tmp/data.2017-02-09T14:14:31.564728


In [3]:
def evaluation(image_path, ckpt_path):
    tf.reset_default_graph()

    jpeg = tf.read_file(image_path)
    image = tf.image.decode_jpeg(jpeg, channels=3)
    image = tf.cast(image, tf.float32)
    image.set_shape([IMAGE_SIZE, IMAGE_SIZE, 3])
    image = tf.image.resize_images(image, [DST_INPUT_SIZE, DST_INPUT_SIZE])
    image = tf.image.per_image_standardization(image)
    image = tf.reshape(image, [-1,DST_INPUT_SIZE * DST_INPUT_SIZE * 3])

    p = inference(image, 1.0)

    sess = tf.InteractiveSession()
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, ckpt_path)

    results = sess.run(p)

    result = results[0]
    rates = [round(n * 100.0, 1) for n in result]

    pred = np.argmax(result)

    ranking = []
    for idx, rate in enumerate(rates):
        name = KINDS[idx]
        ranking.append({
            'image': image_path,
            'kind': name,
            'rate': rate
        })
    rank = sorted(ranking, key=lambda x: x['rate'], reverse=True)

    return (rank, pred)

In [4]:
evaluation('resize/out/test/bags/coordisnap-items-1646784-909236-2qspp7.jpg', './cnn-session-2000')

([{'image': 'resize/out/test/bags/coordisnap-items-1646784-909236-2qspp7.jpg',
   'kind': 'バッグ',
   'rate': 99.799999999999997},
  {'image': 'resize/out/test/bags/coordisnap-items-1646784-909236-2qspp7.jpg',
   'kind': 'スカート',
   'rate': 0.10000000000000001},
  {'image': 'resize/out/test/bags/coordisnap-items-1646784-909236-2qspp7.jpg',
   'kind': 'トップス',
   'rate': 0.10000000000000001},
  {'image': 'resize/out/test/bags/coordisnap-items-1646784-909236-2qspp7.jpg',
   'kind': 'パンツ',
   'rate': 0.0},
  {'image': 'resize/out/test/bags/coordisnap-items-1646784-909236-2qspp7.jpg',
   'kind': 'シューズ',
   'rate': 0.0}],
 0)

In [5]:
import glob

images = glob.glob('resize/out/test/shoes/*.jpg')

success = []
failure = []

for image in images:
    rank, pred = evaluation(image, './cnn-session-2000')
    

    if pred == 2:
        success.append(rank)
    else:
        failure.append(rank)
               
print('success')
print(len(success))
print(success[:10])
print('failure')
print(len(failure))
print(failure[:10])

success
88
[[{'image': 'resize/out/test/shoes/coordisnap-items-1543814-910703-0qrkyc.jpg', 'kind': 'シューズ', 'rate': 100.0}, {'image': 'resize/out/test/shoes/coordisnap-items-1543814-910703-0qrkyc.jpg', 'kind': 'バッグ', 'rate': 0.0}, {'image': 'resize/out/test/shoes/coordisnap-items-1543814-910703-0qrkyc.jpg', 'kind': 'パンツ', 'rate': 0.0}, {'image': 'resize/out/test/shoes/coordisnap-items-1543814-910703-0qrkyc.jpg', 'kind': 'スカート', 'rate': 0.0}, {'image': 'resize/out/test/shoes/coordisnap-items-1543814-910703-0qrkyc.jpg', 'kind': 'トップス', 'rate': 0.0}], [{'image': 'resize/out/test/shoes/coordisnap-items-1646784-909699-6qscuf.jpg', 'kind': 'シューズ', 'rate': 86.799999999999997}, {'image': 'resize/out/test/shoes/coordisnap-items-1646784-909699-6qscuf.jpg', 'kind': 'バッグ', 'rate': 12.699999999999999}, {'image': 'resize/out/test/shoes/coordisnap-items-1646784-909699-6qscuf.jpg', 'kind': 'パンツ', 'rate': 0.40000000000000002}, {'image': 'resize/out/test/shoes/coordisnap-items-1646784-909699-6qscuf.jpg',